## Import libraries

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime

import matplotlib.pyplot as plt
import seaborn as sns

sns.set(style='darkgrid')

## Load and read data

In [ ]:
df = pd.read_csv('./data.csv')
df.head()

In [ ]:
df.shape

In [ ]:
df.info()

## Exploratory Data Analysis (EDA)

### Cleaning data

In [ ]:
df.dropna(subset='Price', inplace=True)
df.isnull().sum()

For columns **Car/Suv**, **BodyType**, the number of null is small, we consider to removing the rows with null values

In [ ]:
dropna_cols = ['Car/Suv', 'BodyType']

df.dropna(subset=dropna_cols, inplace=True)

df.isnull().sum()

#### Location

Since there are 439 null value in **Location** column, removing these rows may result in a significant loss of data. Alternatively, we can fill the null values with a `Unknown` value.

In [ ]:
df['Location'].fillna(value='Unknown', inplace=True)

#### Doors and Seats

Extract the integer value from **Doors** and **Seats** columns, fill null value with mean and round them

In [ ]:
df['Doors'] = df['Doors'].str.extract('(\d+)').astype(float)
df['Seats'] = df['Seats'].str.extract('(\d+)').astype(float)

# fillna with round mean
df['Doors'].fillna(value=round(df['Doors'].mode().iloc[0]), inplace=True)
df['Seats'].fillna(value=round(df['Seats'].mode().iloc[0]), inplace=True)

In [ ]:
df.isnull().sum()

# assert all values are not null
assert df.isnull().sum().sum() == 0

### Check duplicates

In [ ]:
df.duplicated().sum()

# assert no duplicates
assert df.duplicated().sum() == 0

### Convert column data type

In [ ]:
df.info()

Convert the **Kilometres**, **Price** columns to a numerical data type (float) and **Year** column to integer type.

In [ ]:
df['Kilometres'] = pd.to_numeric(df['Kilometres'], errors='coerce')
df["Price"] = pd.to_numeric(df['Price'], errors='coerce')
df['Year'] = df['Year'].astype(int)

In [ ]:
df.info()

Extract the numerical values from the **FuelConsumption**, **CylindersinEngine**, **Engine** column to convert it into a numerical data type (e.g., float). This will allow us to perform calculations or comparisons based on fuel consumption.

In [ ]:
df['FuelConsumption'] = df['FuelConsumption'].str.extract('([\d.]+) L / 100 km').astype(float)
df['CylindersinEngine'] = df['CylindersinEngine'].str.extract('(\d+)').astype(float)
df['Engine'] = df['Engine'].str.extract('([\d.]+)').astype(float)

Replace non-sense value of **Transmission** and **FuelType** with `Other`

In [ ]:
df['Transmission'] = df['Transmission'].replace('-', 'Other')
df['FuelType'] = df['FuelType'].replace('-', 'Other')

In [ ]:
df.head()